In [1]:
import json
import pandas as pd


# Load transcript data

Since this is not a properly-formatted json, we can't load it directly with the json library. We'll have to parse it line by line

In [2]:
with open('./data/raw/transcript.json') as f:
    transaction_list = f.read().splitlines()

In [3]:
# Peek into the first 5 lines
for line in transaction_list[:5]:
    print(line)

{"person": "78afa995795e4d85b5d9ceeca43f5fef", "event": "offer received", "value": {"offer id": "9b98b8c7a33c4b65b9aebfe6a799e6d9"}, "time": 0}
{"person": "a03223e636434f42ac4c3df47e8bac43", "event": "offer received", "value": {"offer id": "0b1e1539f2cc45b7b9fa7c272da2e1d7"}, "time": 0}
{"person": "e2127556f4f64592b11af22de27a7932", "event": "offer received", "value": {"offer id": "2906b810c7d4411798c6938adc9daaa5"}, "time": 0}
{"person": "8ec6ce2a7e7949b1bf142def7d0e0586", "event": "offer received", "value": {"offer id": "fafdcd668e3743c1bb461111dcafc2a4"}, "time": 0}
{"person": "68617ca6246f4fbc85e91a2a49552598", "event": "offer received", "value": {"offer id": "4d5c57ea9a6940dd891ad53e9dbe8da0"}, "time": 0}


In [4]:
def transaction_line_to_dict(line):
    """Parse the json line by line"""
    try:
        transaction = json.loads(line)
        return transaction
    except json.JSONDecodeError:
        pass


In [5]:
# Only load non-blank lines then peek into the first 5 lines
transaction_list = [transaction_line_to_dict(line) for line in transaction_list if line]
for line in transaction_list[:5]:
    print(line)

{'person': '78afa995795e4d85b5d9ceeca43f5fef', 'event': 'offer received', 'value': {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}, 'time': 0}
{'person': 'a03223e636434f42ac4c3df47e8bac43', 'event': 'offer received', 'value': {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}, 'time': 0}
{'person': 'e2127556f4f64592b11af22de27a7932', 'event': 'offer received', 'value': {'offer id': '2906b810c7d4411798c6938adc9daaa5'}, 'time': 0}
{'person': '8ec6ce2a7e7949b1bf142def7d0e0586', 'event': 'offer received', 'value': {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}, 'time': 0}
{'person': '68617ca6246f4fbc85e91a2a49552598', 'event': 'offer received', 'value': {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}, 'time': 0}


If we convert this to a pandas dataframe directly, the `value` column will contain dictionaries, which is not ideal for analysis. Therefore, I'll parse the keys in this column into separate columns.

To do so, we'll need to get all the available keys in that `value` column first so we'll know what columns we want to add.

In [6]:
all_keys = [list(transaction['value']) for transaction in transaction_list]
all_keys = set([item for sublist in all_keys for item in sublist])
print(all_keys)

{'reward', 'offer_id', 'amount', 'offer id'}


Looks like we got `offer_id` and `offer id` which appear to refer to the same thing. To be sure, we'll see if there's any record that has both fields. But first, let's convert the transaction details into a pandas dataframe, which allows us to do quick array manipulations.

In [7]:
def get_transaction_details(transaction):
    # Initialise placeholder for transaction details
    transaction_details = {key: None for key in ('amount', 'offer id', 'offer_id', 'reward')}

    # Get details
    transaction_details.update(transaction)
    transaction_details.update(transaction['value'])

    # Remove value key as details are already parsed
    del transaction_details['value']

    return transaction_details

In [8]:
all_transactions = [
    get_transaction_details(transaction)
    for transaction
    in transaction_list
]

In [9]:
transaction_frame = pd.DataFrame(all_transactions)
transaction_frame.head()

,amount,offer id,offer_id,reward,person,event,time
0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,None,NaN,78afa995795e4d85b5d9ceeca43f5fef,offer received,0
1,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,None,NaN,a03223e636434f42ac4c3df47e8bac43,offer received,0
2,NaN,2906b810c7d4411798c6938adc9daaa5,None,NaN,e2127556f4f64592b11af22de27a7932,offer received,0
3,NaN,fafdcd668e3743c1bb461111dcafc2a4,None,NaN,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0
4,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,None,NaN,68617ca6246f4fbc85e91a2a49552598,offer received,0


Now let's check if there's any row with both `offer_id` and `offer id`

In [10]:
(~transaction_frame[['offer id', 'offer_id']].isnull()).sum(axis=1).value_counts()

1    167581
0    138953
dtype: int64

Looks like the records only have either one of these 2 fields, so let's merge them into 1 column: `offer`

In [11]:
# Merge into 2
transaction_frame['offer'] = transaction_frame['offer_id'].where(
    transaction_frame['offer id'].isnull(),
    transaction_frame['offer id']
)

# Drop the 2
transaction_frame.drop(['offer_id', 'offer id'], axis=1, inplace=True)

In [12]:
transaction_frame.head()

,amount,reward,person,event,time,offer
0,NaN,NaN,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,NaN,NaN,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5
3,NaN,NaN,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4
4,NaN,NaN,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0


Finally, let's save the data for further analysis

In [13]:
transaction_frame.to_csv('./data/processed/transcript.csv', index=False)